# Project final:

Team:
- Kiều Hải Đăng - 19127347
- Lã Minh Hiếu - 19127400

In [1]:
import pandas as pd
import json
import string
import re

## 1. Collecting data:

**1.1 The subject of data:**

- The dataset about TED Talks on the TED.com

**1.2 The origin of data:**
- We get dataset form [kaggle.](https://www.kaggle.com/rounakbanik/ted-talks?select=ted_main.csv)

**1.3 The licence of data:**
- The licence is [CC BY-NC-SA 4.0.](https://creativecommons.org/licenses/by-nc-sa/4.0/)

**1.4 How did authors collect data?**
- The data has been scraped from the official TED Website.

**1.5 Data describtion**
- These datasets contain information of all audio-video recordings of TED Talks uploaded to the official TED.com website until September 21st, 2017. 


## 2. Exploring data

- How many rows and how many columns?
- What is the meaning of each row?

    - Are there **duplicated** rows?
    - What is the meaning of each column?
    - What is the current data type of each column? Are there columns having inappropriate data types?
    
    - With each **numerical** column, how are values distributed?
        - What is the percentage of missing values?
        - Min? max? Are they abnormal?
        
    - With each **categorical** column, how are values distributed?
        - What is the percentage of missing values?
        - How many different values? Show a few
        - Are they abnormal?

In [2]:
ted_df = pd.read_csv('ted_main.csv')

#### How many rows and how many columns?

In [55]:
n_rows, n_cols = ted_df.shape

#### Are there duplicated rows?

In [4]:
ted_df.duplicated().any()

False

#### What is the meaning of each column?

- name: The official name of the TED Talk. Includes the title and the speaker.
- title: The title of the talk
- description: A blurb of what the talk is about.
- main_speaker: The first named speaker of the talk.
- speaker_occupation: The occupation of the main speaker.
- num_speaker: The number of speakers in the talk.
- duration: The duration of the talk in seconds.
- event: The TED/TEDx event where the talk took place.
- film_date: The Unix timestamp of the filming.
- published_date: The Unix timestamp for the publication of the talk on TED.com
- comments: The number of first level comments made on the talk.
- tags: The themes associated with the talk.
- languages: The number of languages in which the talk is available.
- ratings: A stringified dictionary of the various ratings given to the talk (inspiring, fascinating, jaw dropping, etc.)
- related_talks: A list of dictionaries of recommended talks to watch next.
- url: The URL of the talk.
- views: The number of views on the talk.

#### What is the current data type of each column? Are there columns having inappropriate data types?

In [5]:
ted_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2550 entries, 0 to 2549
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   comments            2550 non-null   int64 
 1   description         2550 non-null   object
 2   duration            2550 non-null   int64 
 3   event               2550 non-null   object
 4   film_date           2550 non-null   int64 
 5   languages           2550 non-null   int64 
 6   main_speaker        2550 non-null   object
 7   name                2550 non-null   object
 8   num_speaker         2550 non-null   int64 
 9   published_date      2550 non-null   int64 
 10  ratings             2550 non-null   object
 11  related_talks       2550 non-null   object
 12  speaker_occupation  2544 non-null   object
 13  tags                2550 non-null   object
 14  title               2550 non-null   object
 15  url                 2550 non-null   object
 16  views               2550

- `Inappropriate data types`: film_date, published_date

#### With each numerical column, how are values distributed?

- What is the percentage of missing values?
- Min? max? Are they abnormal?

In [6]:
numerics = ['int16', 'int32', 'int64']
num_df = ted_df.select_dtypes(include=numerics)
num_df.describe()
# not nomal: published_date, film_date

,comments,duration,film_date,languages,num_speaker,published_date,views
count,2550.000000,2550.000000,2.550000e+03,2550.000000,2550.000000,2.550000e+03,2.550000e+03
mean,191.562353,826.510196,1.321928e+09,27.326275,1.028235,1.343525e+09,1.698297e+06
std,282.315223,374.009138,1.197391e+08,9.563452,0.207705,9.464009e+07,2.498479e+06
min,2.000000,135.000000,7.464960e+07,0.000000,1.000000,1.151367e+09,5.044300e+04
25%,63.000000,577.000000,1.257466e+09,23.000000,1.000000,1.268463e+09,7.557928e+05
50%,118.000000,848.000000,1.333238e+09,28.000000,1.000000,1.340935e+09,1.124524e+06
75%,221.750000,1046.750000,1.412964e+09,33.000000,1.000000,1.423432e+09,1.700760e+06
max,6404.000000,5256.000000,1.503792e+09,72.000000,5.000000,1.506092e+09,4.722711e+07


In [7]:
num_df['published_date'] = pd.to_datetime(num_df['published_date'], unit='s')
num_df['film_date'] = pd.to_datetime(num_df['film_date'], unit='s')
num_df

/tmp/ipykernel_134/2115298601.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  num_df['published_date'] = pd.to_datetime(num_df['published_date'], unit='s')
/tmp/ipykernel_134/2115298601.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  num_df['film_date'] = pd.to_datetime(num_df['film_date'], unit='s')


,comments,duration,film_date,languages,num_speaker,published_date,views
0,4553,1164,2006-02-25,60,1,2006-06-27 00:11:00,47227110
1,265,977,2006-02-25,43,1,2006-06-27 00:11:00,3200520
2,124,1286,2006-02-24,26,1,2006-06-27 00:11:00,1636292
3,200,1116,2006-02-26,35,1,2006-06-27 00:11:00,1697550
4,593,1190,2006-02-22,48,1,2006-06-27 20:38:00,12005869
...,...,...,...,...,...,...,...
2545,17,476,2017-06-06,4,1,2017-09-19 20:00:16,450430
2546,6,290,2017-04-24,3,1,2017-09-20 15:02:17,417470
2547,10,651,2017-04-24,1,1,2017-09-21 15:01:35,375647
2548,32,1100,2017-07-08,1,1,2017-09-21 20:00:42,419309


#### With each categorical column, how are values distributed?

- What is the percentage of missing values?
- How many different values? Show a few
- Are they abnormal?

In [36]:
cate_df = ted_df.select_dtypes(include=object)
cate_df.groupby('speaker_occupation').sum()
# abnormal: ratings, related_talks, tags
# Sự phân bố số lượng event qua các năm có đều không. Có gì bất thường không, covid chẳng hạn.
# Nghề nghiệp nào khi thuyết trình sẽ có nhiều lượt view nhất? Lượt view đó có phụ thuộc vào danh tiếng, tiền tài, địa vị của nhân vật đó hay không?

,description,event,main_speaker,name,ratings,related_talks,tags,title,url
speaker_occupation,,,,,,,,,
Chairman of the Cordoba Initiative,Imam Faisal Abdul Rauf combines the teachings ...,TEDSalon 2009 Compassion,Feisal Abdul Rauf,"Feisal Abdul Rauf: Lose your ego, find your co...","[{'id': 11, 'name': 'Longwinded', 'count': 35}...","[{'id': 674, 'hero': 'https://pe.tedcdn.com/im...","['charter for compassion', 'compassion', 'glob...","Lose your ego, find your compassion",https://www.ted.com/talks/imam_feisal_abdul_ra...
"Child protection leader, activist, author",Could it be wrong to help children in need by ...,TEDxSydney,Tara Winkler,Tara Winkler: Why we need to end the era of or...,"[{'id': 10, 'name': 'Inspiring', 'count': 119}...","[{'id': 1596, 'hero': 'https://pe.tedcdn.com/i...","['TEDx', 'activism', 'children', 'family', 'po...",Why we need to end the era of orphanages,https://www.ted.com/talks/tara_winkler_why_we_...
Robotics engineer,Science fiction visions of the future show us ...,TED2017,Radhika Nagpal,Radhika Nagpal: What intelligent machines can ...,"[{'id': 1, 'name': 'Beautiful', 'count': 14}, ...","[{'id': 2346, 'hero': 'https://pe.tedcdn.com/i...","['AI', 'ants', 'fish', 'future', 'innovation',...",What intelligent machines can learn from a sch...,https://www.ted.com/talks/radhika_nagpal_what_...
Space physicist,Is there life beyond Earth? Join NASA's direct...,TED Talks Live,James Green,James Green: 3 moons and a planet that could h...,"[{'id': 8, 'name': 'Informative', 'count': 383...","[{'id': 2227, 'hero': 'https://pe.tedcdn.com/i...","['Mars', 'NASA', 'astrobiology', 'astronomy', ...",3 moons and a planet that could have alien life,https://www.ted.com/talks/james_green_3_moons_...
3D printer,"Just like his beloved grandfather, Avi Reichen...",TED2014,Avi Reichental,Avi Reichental: What's next in 3D printing,"[{'id': 1, 'name': 'Beautiful', 'count': 342},...","[{'id': 1707, 'hero': 'https://pe.tedcdn.com/i...","['manufacturing', 'technology']",What's next in 3D printing,https://www.ted.com/talks/avi_reichental_what_...
...,...,...,...,...,...,...,...,...,...
e-Patient,When Dave deBronkart learned he had a rare and...,TEDxMaastricht,Dave deBronkart,Dave deBronkart: Meet e-Patient Dave,"[{'id': 22, 'name': 'Fascinating', 'count': 11...","[{'id': 484, 'hero': 'https://pe.tedcdn.com/im...","['TEDx', 'collaboration', 'culture', 'data', '...",Meet e-Patient Dave,https://www.ted.com/talks/dave_debronkart_meet...
"inventor, engineer",Fumes from indoor cooking fires kill more than...,TED2006,Amy Smith,Amy Smith: Simple designs to save a life,"[{'id': 9, 'name': 'Ingenious', 'count': 321},...","[{'id': 1561, 'hero': 'https://pe.tedcdn.com/i...","['MacArthur grant', 'alternative energy', 'des...",Simple designs to save a life,https://www.ted.com/talks/amy_smith_shares_sim...
medical inventor,Forget stitches -- there's a better way to clo...,TEDGlobal 2014,Joe Landolina,Joe Landolina: This gel can make you stop blee...,"[{'id': 9, 'name': 'Ingenious', 'count': 808},...","[{'id': 1485, 'hero': 'https://pe.tedcdn.com/i...","['TED Fellows', 'biology', 'biotech', 'health ...",This gel can make you stop bleeding instantly,https://www.ted.com/talks/joe_landolina_this_g...


In [54]:
a = cate_df['ratings'].str.extract(r'\[(.*)\]')
b = a.unstack()
c = b.str.replace('}, ', '}>>')
d = c.str.split('>>')
e = d.droplevel(0)

json_data = []

for i in range(0, len(e)):
    json_data.append([])
    for j in range(0, len(e[i])):
        my_str = e[i][j]
        my_str.replace("'", '"')
        json_data[i].append(js)

/tmp/ipykernel_134/164887740.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  c = b.str.replace('}, ', '}>>')


7

In [10]:
cate_df.isnull().mean() * 100

description           0.000000
event                 0.000000
main_speaker          0.000000
name                  0.000000
ratings               0.000000
related_talks         0.000000
speaker_occupation    0.235294
tags                  0.000000
title                 0.000000
url                   0.000000
dtype: float64

In [11]:
ted_df.nunique(axis=0)

comments               559
description           2550
duration              1083
event                  355
film_date              735
languages               66
main_speaker          2156
name                  2550
num_speaker              5
published_date        2490
ratings               2550
related_talks         2550
speaker_occupation    1458
tags                  2530
title                 2550
url                   2550
views                 2550
dtype: int64

## 3. Asking meaningful questions

- What is the question?
- What are benefits of finding the answer?

1. Thời gian trung bình của một video là bao nhiêu? Độ dài ngắn của một video có ảnh hưởng đến lượt view hay không? Nếu video càng ngắn, lượt xem càng nhiều và các video đều dưới một ngưỡng thời gian nào đó (VD: 30 phút) thì có phải đó là ý đồ của TED nhằm mục đích thu hút người dùng của họ hay không?
2. Video có lượt rate cao nhất? Có phải rating càng nhiều or càng cao thì view càng nhiều hay không. Video càng cũ rating càng cao phải không?
3. Tương quan giữa lượt xem và lượt bình luận của mỗi video, có phải đại đa số người dùng không thích or thích chia sẻ cảm nghĩ or góp ý với mỗi video?
4. Người dùng thường có xu hướng related_talk cùng thể loại/tác giả/thời gian/... hay là khác? Đưa ra quan điểm của nhóm giải thích cho vấn đề này.
5. Nghề nghiệp nào khi thuyết trình sẽ có nhiều lượt view nhất? Lượt view đó có phụ thuộc vào danh tiếng, tiền tài, địa vị của nhân vật đó hay không?

In [48]:
a = ted_df.copy()
a.groupby('speaker_occupation').sum().sort_values('views', ascending=False)
# view, tags, comments, ratings, num event (TIME).

a[a['speaker_occupation'].isin(['Economist'])]

,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,url,views
27,95,"""Freakonomics"" author Steven Levitt presents n...",1275,TED2004,1077840000,25,Steven Levitt,Steven Levitt: The freakonomics of crack dealing,1,1158624660,"[{'id': 7, 'name': 'Funny', 'count': 485}, {'i...","[{'id': 20, 'hero': 'https://pe.tedcdn.com/ima...",Economist,"['business', 'cities', 'culture', 'economics',...",The freakonomics of crack dealing,https://www.ted.com/talks/steven_levitt_analyz...,2863214
110,127,We know the negative images of Africa -- famin...,1213,TED2007,1172880000,32,Ngozi Okonjo-Iweala,Ngozi Okonjo-Iweala: Want to help Africa? Do b...,1,1180491120,"[{'id': 1, 'name': 'Beautiful', 'count': 132},...","[{'id': 185, 'hero': 'https://pe.tedcdn.com/im...",Economist,"['business', 'corruption', 'global issues', 'i...",Want to help Africa? Do business here,https://www.ted.com/talks/ngozi_okonjo_iweala_...,1044185
127,97,Ghanaian economist George Ayittey unleashes a ...,1070,TEDGlobal 2007,1181001600,20,George Ayittey,George Ayittey: Africa's cheetahs versus hippos,1,1185791520,"[{'id': 8, 'name': 'Informative', 'count': 249...","[{'id': 152, 'hero': 'https://pe.tedcdn.com/im...",Economist,"['Africa', 'business', 'corruption', 'economic...",Africa's cheetahs versus hippos,https://www.ted.com/talks/george_ayittey_on_ch...,648234
128,108,"Ngozi Okonjo-Iweala, the former finance minist...",1330,TEDGlobal 2007,1181174400,21,Ngozi Okonjo-Iweala,Ngozi Okonjo-Iweala: Aid versus trade,1,1185879300,"[{'id': 3, 'name': 'Courageous', 'count': 102}...","[{'id': 91, 'hero': 'https://pe.tedcdn.com/ima...",Economist,"['Africa', 'business', 'economics', 'entrepren...",Aid versus trade,https://www.ted.com/talks/ngozi_okonjo_iweala_...,524049
155,64,Economist Eleni Gabre-Madhin outlines her ambi...,1234,TEDGlobal 2007,1181088000,21,Eleni Gabre-Madhin,Eleni Gabre-Madhin: A commodities exchange for...,1,1193274000,"[{'id': 8, 'name': 'Informative', 'count': 222...","[{'id': 156, 'hero': 'https://pe.tedcdn.com/im...",Economist,"['business', 'economics', 'food', 'global issu...",A commodities exchange for Ethiopia,https://www.ted.com/talks/elene_gabre_madhin_o...,562457
234,139,"Around the world right now, one billion people...",1011,TED2008,1204502400,23,Paul Collier,"Paul Collier: The ""bottom billion""",1,1211986800,"[{'id': 24, 'name': 'Persuasive', 'count': 443...","[{'id': 157, 'hero': 'https://pe.tedcdn.com/im...",Economist,"['Africa', 'activism', 'business', 'economics'...","The ""bottom billion""",https://www.ted.com/talks/paul_collier_shares_...,990220
245,119,Steven Levitt shares data that shows car seats...,1138,TEDGlobal 2005,1121126400,25,Steven Levitt,Steven Levitt: Surprising stats about child ca...,1,1214269200,"[{'id': 9, 'name': 'Ingenious', 'count': 66}, ...","[{'id': 29, 'hero': 'https://pe.tedcdn.com/ima...",Economist,"['business', 'cars', 'culture', 'economics', '...",Surprising stats about child carseats,https://www.ted.com/talks/steven_levitt_on_chi...,749952
435,202,"The ""dismal science"" truly shines in this opti...",873,TED2009,1233964800,33,Alex Tabarrok,Alex Tabarrok: How ideas trump crises,1,1240794000,"[{'id': 10, 'name': 'Inspiring', 'count': 267}...","[{'id': 63, 'hero': 'https://pe.tedcdn.com/ima...",Economist,"['business', 'china', 'cities', 'culture', 'ec...",How ideas trump crises,https://www.ted.com/talks/alex_tabarrok_forese...,748739
476,93,"Long conflict can wreck a country, leaving beh...",994,TED@State,1244246400,28,Paul Collier,Paul Collier: New rules for rebuilding a broke...,1,1245805200,"[{'id': 8, 'name': 'Informative', 'count': 162...","[{'id': 270, 'hero': 'https://pe.tedcdn.com/im...",Economist,"['global issues', 'money', 'politics', 'povert...",New rules for rebuilding a broken nation,https://www.ted.com/talks/paul_collier_s_new_r...,406530
782,364,"As the world faces recession, climate change, ...",1223,TEDGl

## 4. Preprocessing + analyzing data to answer each question

In [12]:
raise NotImplementedError()

NotImplementedError: 

## 5. Conclusion

## 6. Reflection

- Each member: What difficulties have you encountered?
- Each member: What have you learned?
- Your group: If you had more time, what would you do?

## 7. Reference
1. [Data set](https://www.kaggle.com/rounakbanik/ted-talks).
2. https://gocnhintangphat.com/timestamp-la-gi/.
3. https://rstudio-pubs-static.s3.amazonaws.com/321337_38458c80a3fb4edf8755e8bce876e822.html.
4. https://stackoom.com/en/question/3w6dO.
5. https://www.guru99.com/python-json.html.
6. https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html.
7. https://pretagteam.com/question/how-to-replace-special-characters-in-a-string-python.